# TFT Simulation

In [1]:
from lib import *
#auto100 for tft growth means that the tft has the same value of 100% of the value of the grid based on 10x margin over 5Y
#node_growth="0:5,6:150,20:2000"
simulation = j.tools.tfgrid_simulator.simulation_get(
            name="default",
            tokencreator_name="optimized",
            hardware_config_name="amd",
            node_growth="0:5,6:150,20:2000",
            tft_growth=3,
            reload=True,
        )
server=simulation.environment.node_normalized
md_code("params/default.py")

```python
params/default.py
```

In [2]:
md(f"""
## Simulation Arguments 

- nr of nodes at start: {simulation.nodebatches[0].nrnodes}
- nr of months left (avg out) for farming of startingnodes: {simulation.nodebatches[0].months_left}
- nr of TFT already farmed: { j.data.types.numeric.clean(simulation.nodebatches[0].tft_farmed_before_simulation)} 

""")


## Simulation Arguments 

- nr of nodes at start: 1500
- nr of months left (avg out) for farming of startingnodes: 36
- nr of TFT already farmed: 800,000,000 



## Device Details

In [3]:

md(f"""
### Device Details

- nr of TB netto usable = {round(server.su,1)} TB
- nr of cpu cores = {round(server.cru,1)} Cores
- power used = {server.power} watt
- performance in passmark = {round(server.cu_passmark*server.cu)}

""")


### Device Details

- nr of TB netto usable = 58.3 TB
- nr of cpu cores = 16.0 Cores
- power used = 222 watt
- performance in passmark = 24479



## Growth per month

In [4]:
simulation.graph_nr_nodes()

FigureWidget({
    'data': [{'name': 'new nodes',
              'type': 'scatter',
              'uid': 'f16d9…

### Number of racks per month


In [5]:
x=np.arange(0,60)
cells= [int(i/44) for i in simulation.sheet.rows.rackspace_u.cells[0:60]]
fig = go.FigureWidget(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'Racks Occupied per Month',showlegend=False)
# fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '1c4bdb06-f1ec-4a41-a3fb-0af575c78d67',
 …

In [6]:
cells= [int(i) for i in simulation.sheet.rows.power_kw.cells[0:60]]
fig = go.FigureWidget(data=go.Scatter(x=x, y=cells))
fig.update_layout(title = 'Kwatt/h per Month used.',showlegend=False)
fig

FigureWidget({
    'data': [{'type': 'scatter',
              'uid': 'b022add4-6994-4391-ba1a-c6233e6598b1',
 …

## TFT Growth

In [7]:
names=["farmed","cultivated","sold","burned","farmer_income"]
def do(end):
    fig = go.FigureWidget()
    title = 'TFT Farmed, Cultivated, Sold, Burned per month, for batch month:%s.'%end
    xx=[i for i in range(1,end+1)]
    for i in range(5):
        name=names[i]
        cells=simulation.rows["tft_%s"%name].cells[1:end+1]
        fig.add_trace(go.Scatter(x=xx, y=cells,name=name))
    fig.update_layout(title = title,showlegend=True)
    return fig


In [8]:
do(12)

FigureWidget({
    'data': [{'name': 'farmed',
              'type': 'scatter',
              'uid': '65cf1098…

In [9]:
do(30)

FigureWidget({
    'data': [{'name': 'farmed',
              'type': 'scatter',
              'uid': '56346a82…

In [10]:
do(60)

FigureWidget({
    'data': [{'name': 'farmed',
              'type': 'scatter',
              'uid': 'eebe7623…

In [11]:
fig1, fig2, fig3, fig4, fig5, fig6, fig7,fig8=simulation.graph_tft_simulation(show=False);

In [12]:
fig1

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'nrnodes_total',
              'type'…

In [13]:
fig2

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'tft_farmed',
              'type': '…

In [14]:
fig8

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'tft_marketcap',
              'type'…

In [15]:
fig3

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'tft_farmed_cumul',
              'ty…

In [16]:
fig4


FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'USD',
              'type': 'scatter…

In [17]:
fig5

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'multiple',
              'type': 'sc…

In [18]:
fig6

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'USD',
              'type': 'scatter…

In [19]:
fig7

FigureWidget({
    'data': [{'connectgaps': False,
              'name': 'multiple',
              'type': 'sc…

## Token price per month


In [20]:
simulation.graph_token_price()


FigureWidget({
    'data': [{'type': 'scatter',
              'uid': '2432d63b-2d91-4665-88be-7d6297eebd45',
 …

In [21]:
# %qtconsole